In [254]:
%matplotlib qt
import scipy.stats as st
import numpy as np
from scipy import fftpack
import scipy.ndimage.filters
import matplotlib.pyplot as plt
import skimage.io
import skimage.exposure
import skimage.util
import skimage.morphology
import scipy.ndimage
from skimage.measure import compare_ssim
import cv2

In [255]:
filePath = './strb1.jpg';

In [256]:
img = cv2.imread(filePath)
cv2.imwrite('1_out_initial.jpg', img)

True

In [257]:
# Blur the red image
ksize = (5, 5) 
blurredImage = cv2.blur(img, ksize, cv2.BORDER_DEFAULT) 
cv2.imwrite('2_blurred3_red.jpg', np.hstack([img, blurredImage]))

True

In [258]:
# Isolate very bright red colors ( classical WAY )
redBoundary = ([0, 0, 100], [200, 200, 255])
maskRed = cv2.inRange(blurredImage, np.array(redBoundary[0],dtype = "uint8"), np.array(redBoundary[1], dtype = "uint8"))
outputRed = cv2.bitwise_and(blurredImage, blurredImage, mask = maskRed)

greenBoundary = ([0, 100, 0], [200, 255, 200])
maskGreen = cv2.inRange(blurredImage, np.array(greenBoundary[0],dtype = "uint8"), np.array(greenBoundary[1], dtype = "uint8"))
outputGreen = cv2.bitwise_and(blurredImage, blurredImage, mask = maskGreen)

blueBoundary = ([100, 0, 0], [200, 255, 200])
maskBlue = cv2.inRange(blurredImage, np.array(blueBoundary[0],dtype = "uint8"), np.array(blueBoundary[1], dtype = "uint8"))
outputBlue = cv2.bitwise_and(blurredImage, blurredImage, mask = maskBlue)

cv2.imwrite("3_red_green_blue.jpg", np.hstack([outputRed, outputGreen, outputBlue]))
cv2.imwrite("3_reconstructed.jpg", cv2.add(outputRed, cv2.add(outputGreen, outputBlue)))
onlyRedMask =  np.array(np.int32(maskRed) -  np.int32(maskGreen) -  np.int32(maskBlue))
onlyRedMask = np.array(onlyRedMask.clip(0, 255), dtype = "uint8")
onlyRedImage = cv2.bitwise_and(img, img, mask = onlyRedMask)
cv2.imwrite('3_only_red.jpg', onlyRedImage)
cv2.imwrite('3_raw_only_red.jpg', np.hstack([img, onlyRedImage]))

True

In [259]:
hsvImage = cv2.cvtColor(blurredImage, cv2.COLOR_BGR2HSV)

#For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255].
# Filter by colour
# 0-10 hue
# minimum red amount, max red amount
min_red = np.array([0, 100, 80])
max_red = np.array([10, 256, 256])
# layer
mask1 = cv2.inRange(hsvImage, min_red, max_red)
cv2.imwrite('3_hsv_mask1.jpg', mask1)
# birghtness of a color is hue
# 170-180 hue
min_red2 = np.array([170, 100, 80])
max_red2 = np.array([180, 256, 256])
mask2 = cv2.inRange(hsvImage, min_red2, max_red2)
cv2.imwrite('3_hsv_mask2.jpg', mask2)

# looking for what is in both ranges
# Combine masks
hsvMask = mask1 + mask2
cv2.imwrite('3_hsv_mask3.jpg', np.hstack([img, cv2.bitwise_and(img, img, mask = hsvMask)]))
#

True

In [260]:
# Do erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))

# morph the image. closing operation Dilation followed by Erosion.
# It is useful in closing small holes inside the foreground objects,
# or small black points on the object.
mask_closed = cv2.morphologyEx(hsvMask, cv2.MORPH_CLOSE, kernel)
# erosion followed by dilation. It is useful in removing noise
mask_clean = cv2.morphologyEx(mask_closed, cv2.MORPH_OPEN, kernel)

onlyRedImageDirtyHsv = cv2.bitwise_and(img, img, mask = hsvMask)
onlyRedImageCleanHsv = cv2.bitwise_and(img, img, mask = mask_clean)
cv2.imwrite('3_hsv_mask.jpg', np.hstack([onlyRedImageDirtyHsv, onlyRedImageCleanHsv]))

True

In [261]:
# Make image gray scaled
imgGrey = cv2.cvtColor(np.array(onlyRedImageCleanHsv, dtype = "uint8"), cv2.COLOR_BGR2GRAY)
cv2.imwrite('4_grayscale.jpg', imgGrey)

True

In [262]:
# Binarize with otsu
threshold, otsuImage = cv2.threshold(imgGrey, 0, 255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('5_otsu.jpg', np.hstack([img, cv2.cvtColor(otsuImage,cv2.COLOR_GRAY2RGB)]))

True

In [263]:
# Build distance transform
distanceTransformImage = cv2.distanceTransform(otsuImage,cv2.DIST_L2, 5)
cv2.imwrite('6_distance_transform.jpg', distanceTransformImage)
        

True

In [264]:
# Get maximum local points
maximumLocalPoints = []
for x in range(distanceTransformImage.shape[0]):
    for y in range(distanceTransformImage.shape[1]):
        dx = [-1, 1, 0, 0, -1, -1, 1, 1]
        dy = [0, 0, -1, 1, -1, 1, -1, 1]
        
        isMaximumLocal = True
        for toGo in range(len(dx)):
            newX = x + dx[toGo]
            newY = y + dy[toGo]
            if newX < 0 or newY < 0 or newX >= distanceTransformImage.shape[0] or newY >= distanceTransformImage.shape[1]:
                continue
                
            if distanceTransformImage[x][y] <= distanceTransformImage[newX][newY]:
                isMaximumLocal = False
                break
            
        if isMaximumLocal:
            maximumLocalPoints.append( (x, y) )
            

print(len(maximumLocalPoints))

113


In [265]:
# Do the flooding [NOT WORKING]
fillImage = otsuImage.copy()
h, w =  otsuImage.shape[:2]
fillMask = np.zeros((h + 2, w + 2), np.uint8)

for pointPair in maximumLocalPoints[:2]:
    try:
        cv2.floodFill(fillImage, fillMask, pointPair, 100)
        contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    except:
        pass


fillImageInv = cv2.bitwise_not(fillImage)
cv2.imwrite('7_fill_image.jpg', fillImageInv)
print("done")

done


In [272]:
# Canny Edge
edgedImage = cv2.Canny(imgGrey, 30, 200)
cv2.imwrite('8_canny_edge.jpg', np.hstack([img, cv2.cvtColor(edgedImage,cv2.COLOR_GRAY2RGB)]))

True

In [267]:
# Find Contours after Canny Edge
contoursImage = edgedImage.copy()
contours, hierarchy = cv2.findContours(contoursImage,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 

finalImage = img.copy()
cv2.drawContours(finalImage, contours, -1, (0, 255, 0), 3) 
cv2.imwrite('9_contours.jpg', finalImage)

True

In [268]:
# Merge contours into rects
nuclei = []
rects = []
rects4corners = []
for cnt in contours:   
    if cv2.contourArea(cnt) >= 10:   
        nuclei.append(cnt)
        rect = cv2.boundingRect(cnt)
        rects.append(rect)
        rects4corners.append([rect[0], rect[1], rect[0] + rect[2], rect[1] + rect[3]])

finalImage = img.copy()
try:
    for (x,y,w,h) in rects:
        cv2.rectangle(finalImage,(x,y),(x+w,y+h),(0,0,255),2)
except:
    print("da")
print(len(rects))
cv2.imwrite('9_rects_contours.jpg', finalImage)
finalImageAllRects = finalImage.copy()

6


In [269]:
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")

rectsFiltered = non_max_suppression_fast(np.array(rects4corners), 0.2)
print(len(rectsFiltered))
finalImage = img.copy()
try:
    rectsFiltered = rectsFiltered[1:]
    for (x1,y1,x2,y2) in rectsFiltered:
        cv2.rectangle(finalImage,(x1,y1),(x2, y2),(0,0,255),2)
except:
    pass
cv2.imwrite('10_rects_filtered.jpg', finalImage)

cv2.imwrite('10_rects_all_filtered.jpg', np.hstack([finalImageAllRects, finalImage]))

3


True

In [270]:
cv2.imwrite('11_final.jpg', np.hstack([img, onlyRedImageCleanHsv, finalImage]))
print("Done")

Done
